# Introduction to metabarcoding: DADA2 Tutorial

**Workshop led by: (Pedro E. Romero)** <br>
**Assisted by: Boris ..., Camila Castillo-Vilcahuaman** <br>
**Tutorial by: @reymonera (Camila Castillo-Vilcahuaman)**

This is a tutorial with all the necessary commands for the use of the R package DADA2. All the analysis are done and exposed here with their respective figures. In this opportunity, we will use data from an already published study: Metabarcoding in the most important river in Lima, Peru: The Rimac river.

This pipeline and tutorial is heavily based on another great bioinformatician's pipeline (Hi, @AstroBioMike!). If you want to learn more about this package, I highly suggest to visit this tutorial.

## 1. Getting to know the DADA2 package

First of all, we will get a first glance of how this environment stuff works. First of all, this is a Jupyter notebook. It is easier to show you guys how code works using these.
However, for our workshop to work well, we will need to set everything up first. So, for this, we will proceed with the following code:

```
conda env create -f dada2_env.yaml
conda activate dada2_env
python -m ipykernel install --user --name=dada2_env
jupyter-notebook
```

Now, we will proceed with enabling R code in this notebook. As you've probably guessed, Jupyer mainly runs with python. But we can make our own stuff run here using the following command:

In [1]:
%load_ext rpy2.ipython

/home/marlen/miniforge3/envs/dada2_env/lib/python3.13/site-packages/rpy2/ipython/rmagic.py:85: UserWarning: The Python package `pandas` is strongly recommended when using `rpy2.ipython`. Unfortunately it could not be loaded, as we did not manage to load `numpy` in the first place (error: No module named 'numpy').
  warnings.warn('The Python package `pandas` is strongly '


## 2. Getting to know our data

Empezamos por descargar la información necesaria para nuestro análisis. Estaremos trabajando con la data del estudio de las comunidades microbianas presentes en el río Rímac. Para ello requeriremos descargarlas de la siguiente forma: 
- Primero, nos dirigimos a la pestaña “Terminal” de nuestro RStudio. Luego, pegaremos los siguientes comandos:

In [ ]:
!cd ~
!curl -L -o dada2_amplicon_ex_workflow.tar.gz
!https://ndownloader.figshare.com/files/28773936
!tar -xzvf dada2_amplicon_ex_workflow.tar.gz
!rm dada2_amplicon_ex_workflow.tar.gz
!cd dada2_amplicon_ex_workflow/

Con el último comando, nos podremos ubicar dentro de la carpeta. Si le damos a `ls` podremos observar los nuevos archivos que hemos adquirido.

- Ahora, requeriremos un listado de los archivos que tiene por nombre `samples` con la información de nombres de cada uno de los archivos. Esto hará más simple el moverse por los archivos, y facilitará varios aspectos relacionados al procesamiento. Estaremos llamando a estos archivos constantemente. Quedándonos en “Terminal”, ejecutamos el siguiente comando:

In [ ]:
!ls *_R1.fq | cut -f1 -d "_" > samples

- Toca hacerle un tratamiento a nuestros archivos de reads. Ello implica cortar los adaptadores con los que vienen por default. Eso lo podemos lograr con `cutadapt`. En nuestra terminal, ejecutamos el siguiente comando:

In [ ]:
!cutadapt --version # 2.3
!cutadapt -a ^GTGCCAGCMGCCGCGGTAA...ATTAGAWACCCBDGTAGTCC \
-A ^GGACTACHVGGGTWTCTAAT...TTACCGCGGCKGCTGGCAC \
-m 215 -M 285 --discard-untrimmed \
-o B1_sub_R1_trimmed.fq -p B1_sub_R2_trimmed.fq \
B1_sub_R1.fq B1_sub_R2.fq

Y ejecutaremos `cutadapt` para B1. Si deseamos ejecutar cutadapt para el resto de nuestras
muestras, lo mejor será utilizar un artilugio del lenguaje Bash:

In [ ]:
!for sample in $(cat samples)
!do
    !echo "On sample: $sample"

    !cutadapt -a ^GTGCCAGCMGCCGCGGTAA...ATTAGAWACCCBDGTAGTCC \
    -A ^GGACTACHVGGGTWTCTAAT...TTACCGCGGCKGCTGGCAC \
    -m 215 -M 285 --discard-untrimmed \
    -o ${sample}_sub_R1_trimmed.fq.gz -p ${sample}_sub_R2_trimmed.fq.gz \
    ${sample}_sub_R1.fq ${sample}_sub_R2.fq \
    >> cutadapt_primer_trimming_stats.txt 2>&1
!done

Esto terminará por realizar el tratamiento en nuestra data. Ahora sí, podremos dirigirnos a la consola de R y empezar a poner todo en orden. Primero, señalamos en donde nos vamos a quedar y cuál será nuestro espacio de trabajo:

## 3. Starting with DADA2

In [2]:
%%R
getwd()
setwd("~/dada2_amplicon_ex_workflow")

[1] "/home/marlen/gitrepos/dada2_msm-course-2024"


In [10]:
%%R
library(dada2)
packageVersion("dada2")

[1] ‘1.30.0’


Crea una variable llamada `samples`

In [ ]:
%%R
samples <- scan("samples", what="character")
list.files()

Crea dos variables para los archivos filtrados

In [ ]:
%%R
#Iremos creando las variables para conservar los reads filtrados
filtered_forward_reads <- paste0(samples, "_sub_R1_filtered.fq.gz")
filtered_reverse_reads <- paste0(samples, "_sub_R2_filtered.fq.gz")

Crea dos variables `lecturas_directas(forward_reads)` y `lecturas_reversas(reverse_reads)` añadiendo etiquetas _R1 o _R2 a la variable muestras, de modo que concuerde con los nombres de archivo

In [ ]:
%%R
forward_reads <- paste0(samples, "_sub_R1_trimmed.fq.gz")
reverse_reads <- paste0(samples, "_sub_R2_trimmed.fq.gz")

#Iremos creando las variables en donde entrarán los reads ya “podados”
trimmed_forward_reads <- paste0(samples, "_1_trimmed.fastq")
trimmed_reverse_reads <- paste0(samples, "_2_trimmed.fastq")

#A matrix with no blanks included
trimmed_forward_reads[1:25] -> trimmed_forward_reads_noblanks
trimmed_reverse_reads[1:25] -> trimmed_reverse_reads_noblanks

### Quality Plots 

Para saber en donde vamos a cortar, necesitaremos primero ver gráficos de calidad. Eso lo lograremos mediante el comando que le solicita a DADA2 un perfil de calidad, mediante los siguientes comandos:

In [ ]:
%%R

plotQualityProfile(forward_reads)
plotQualityProfile(reverse_reads)
plotQualityProfile(reverse_reads[17:20])

Revisemos el gráfico de calidad, la longitud de lectura es 250 nt. Observemos más o menos en qué punto es que se da la caída en calidad en nuestra data. Esto nos ayudará a los parámetros a continuación.

Luego, utilizaremos el comando `filterAndTrim` que nos permite editar los reads de acuerdo a nuestras necesidades:

In [ ]:
%%R

filtered_out <- filterAndTrim(forward_reads, filtered_forward_reads,
reverse_reads, filtered_reverse_reads, maxEE=c(2,2),
rm.phix=TRUE, minLen=175, truncLen=c(250,200))

filtered_out

#Podemos explorar nuestra data con:
class(filtered_out)
dim(filtered_out)

Vale la pena ahora chequear si es que nuestro filtro realmente valió la pena o no. Para ello, podemos volver a dibujar la data que tenemos disponible:

In [ ]:
%%R

plotQualityProfile(filtered_forward_reads)
plotQualityProfile(filtered_reverse_reads)
plotQualityProfile(filtered_reverse_reads[17:20])

¿Tuvo algún efecto lo cortado y filtrado?

### Learning from our errors

Todo secuenciamiento no está libre de errores. Y DADA2 desea conocerlos. Este es un paso únicamente necesario para nuestro paquete. De aquí la información que nos interesa es poca, pero de todos modos es un paso importante (y computacionalmente intensivo):

In [ ]:
%%R

err_forward_reads <- learnErrors(filtered_forward_reads)
err_reverse_reads <- learnErrors(filtered_reverse_reads)

Finalmente procedemos a dibujar nuestras gráficas de error de la siguiente forma:

In [ ]:
%%R

plotErrors(err_forward_reads, nominalQ=TRUE)
plotErrors(err_reverse_reads, nominalQ=TRUE)

Y sale... esto:

Ahm... no hay mucho que hacer por aquí. Nuevamente, aprender los errores es algo que se necesitará en comandos posteriores. En nuestro caso, lo único importante es que los puntos negros sigan a la curva negra. Eso nos indica la asociación entre observado y estimado.

### Derreplication

¿Queremos gastar nuestros recursos tratando de determinar a trillones de secuencias idénticas?

¿No sería mejor poner la secuencia e indicar la cantidad de veces que anda por ahí? 

Ok, entonces utilizaremos el comando que nos permite ello:

In [ ]:
%%R

derep_forward <- derepFastq(filtered_forward_reads, verbose=TRUE)
names(derep_forward) <- samples

derep_reverse <- derepFastq(filtered_reverse_reads, verbose=TRUE)
names(derep_reverse) <- samples

### Inferring ASVs with the DADA algorithm

Ahora sí, pasemos a los ASVs. Habíamos hablado mucho de estos, pero ya era hora de utilizar un comando para poder tenerlos en nuestras pantallas. Así es cómo utilizamos lo siguiente:

In [ ]:
%%R

dada_forward <- dada(derep_forward, err=err_forward_reads,
pool="pseudo")

dada_reverse <- dada(derep_reverse, err=err_reverse_reads,
pool="pseudo")

El último parámetro `pool` es difícil de explicar, pero podemos decir que las dos opciones eran o chequear muestra por muestra o coger todas las muestras en lo que se conoce como “pooling”. Sin embargo, esto podía terminar obviando algunas secuencias que se perderían en la “piscina” de opciones. 

Entonces, se creó el *pseudo-pooling*. A continuación, uniremos a las secuencias forward y reverse, ya que esto será nuestra secuencia
blanco para poder realizar la identificación de las mismas. Es lo lograremos con los siguientes comandos:

In [ ]:
%%R

merged_amplicons <- mergePairs(dada_forward, derep_forward,
dada_reverse, derep_reverse, trimOverhang=TRUE, minOverlap=170)

Podemos explorar un poco esta variable mediante estos comandos:

In [ ]:
%%R

class(merged_amplicons)
length(merged_amplicons)
names(merged_amplicons)

class(merged_amplicons$B1)
names(merged_amplicons$B1)

### ASVs count table

Quizá el output más importante sea la tabla que se obtiene, a la que mundanamente se le conoce como la tabla de los counts. Esta la podemos producir de la siguiente manera:

In [ ]:
%%R

seqtab <- makeSequenceTable(merged_amplicons)
class(seqtab) # matrix
dim(seqtab) # 20 2521

### Chimera identification

Por último, identificaremos a las quimeras. Estas son secuencias que se pegan sin ningún significado biológico concreto. DADA2 realiza esta identificación haciendo estimaciones entre las secuencias más abundantes y las menos, tratando de inferir si es que se pudieron juntar de alguna forma.

In [ ]:
%%R

seqtab.nochim <- removeBimeraDenovo(seqtab, verbose=T)

#Y podemos revisar cuánto hemos perdido con esto también. La idea es
que tengamos un decimal cercano al 1:
sum(seqtab.nochim)/sum(seqtab)

Finalmente, para finalizar este paso, es adecuado tener una tabla resumen de todo lo que hemos realizado:

In [ ]:
%%R

getN <- function(x) sum(getUniques(x))
summary_tab <- data.frame(row.names=samples,
dada2_input=filtered_out[,1], filtered=filtered_out[,2],
dada_f=sapply(dada_forward, getN), dada_r=sapply(dada_reverse, getN),
merged=sapply(merged_amplicons, getN), nonchim=rowSums(seqtab.nochim),
final_perc_reads_retained=round(rowSums(seqtab.nochim)/filtered_out[,1]
*100, 1))

summary_tab

write.table(summary_tab, "read-count-tracking.tsv", quote=FALSE,
sep="\t", col.names=NA)

## 4. Assigning taxonomy

Normalmente, para poder realizar este paso requeriríamos la descarga la base de datos 16S rRNA [SILVA] (https://zenodo.org/record/3731176#.YJa8K-hKg2w). Sin embargo, las capacidades nos limitan un poco al respecto de esto. Lo mejor que puedo hacer es dejar un código, el cuál es mejor no ejecutar:

In [ ]:
%%R
## skipping this codeblock for time, and it will not run in the binder environment
## downloading DECIPHER-formatted SILVA v138 reference
#download.file(url="http://www2.decipher.codes/Classification/TrainingSets/SILVA_SSU_r138_2019.RData", destfile="SILVA_SSU_r138_2019.RData")

## loading reference taxonomy object
# load("SILVA_SSU_r138_2019.RData")
## loading DECIPHER
# library(DECIPHER)
# packageVersion("DECIPHER") # v2.6.0 when this was initially put together, though might be different in the binder or conda installation, that's ok!

## creating DNAStringSet object of our ASVs
# dna <- DNAStringSet(getSequences(seqtab.nochim))
## and classifying
# tax_info <- IdTaxa(test=dna, trainingSet=trainingSet, strand="both", processors=NULL)

Como no tenemos estas capacidades, utilizaremos lo siguiente, que es data ya estimada por el autor y que nos permite acceder a este análisis desde ya:

In [ ]:
%%R
load("tax-info.RData")

### Taxa-information tables

Normalmente en este tipo de estudios requerimos tablas de secuencias, conteo y taxonomía. Son formatos estándares que son posibles de exportar a otros paquetes. Esto lo podemos conseguir con los siguientes comandos:

In [ ]:
%%R

# asignando nuevos nombres (ASV_1, ASV_2...)
asv_seqs <- colnames(seqtab.nochim)
asv_headers <- vector(dim(seqtab.nochim)[2], mode="character")

for (i in 1:dim(seqtab.nochim)[2]) {
asv_headers[i] <- paste(">ASV", i, sep="_")
}

# escribiendo fastas de nuestros ASVs
asv_fasta <- c(rbind(asv_headers, asv_seqs))
write(asv_fasta, "ASVs.fa")

# Tabla de conteo:
asv_tab <- t(seqtab.nochim)
row.names(asv_tab) <- sub(">", "", asv_headers)
write.table(asv_tab, "ASVs_counts.tsv", sep="\t", quote=F, col.names=NA)

# Tabla de taxas:
# Crear tabla de taxonomías y diciendo que los que no existen son NA
ranks <- c("domain", "phylum", "class", "order", "family", "genus", "species")

asv_tax <- t(sapply(tax_info, function(x) {
    m <- match(ranks, x$rank)
    taxa <- x$taxon[m]
    taxa[startsWith(taxa, "unclassified_")] <- NA
    taxa
}))

colnames(asv_tax) <- ranks
rownames(asv_tax) <- gsub(pattern=">", replacement="", x=asv_headers)

write.table(asv_tax, "ASVs_taxonomy.tsv", sep = "\t", quote=F, col.names=NA)

### Identifying contaminants

Algo muy importante en este tipo de estudios es tener blancos. Muestras sin nada que nos puedan decir si es que hay contaminantes en nuestras propias muestras. De este modo, si especie 1 se encontró en el blanco, es un trabajo más simple eliminar a especie 1 de las muestras. Este descarte lo podemos hacer con el siguiente paquete:

In [ ]:
%%R

library(decontam)
packageVersion("decontam")

colnames(asv_tab) # en estas muestras, los blancos son los primeros 4

vector_for_decontam <- c(rep(TRUE, 4), rep(FALSE, 16))
contam_df <- isContaminant(t(asv_tab), neg=vector_for_decontam)
table(contam_df$contaminant)

contam_asvs <- row.names(contam_df[contam_df$contaminant == TRUE, ])

Ajá, identificados. Ahora, a removerlos y realizar conteos sin contaminantes:

In [ ]:
%%R

# haciendo un nuevo archivo fasta
contam_indices <- which(asv_fasta %in% paste0(">", contam_asvs))
dont_want <- sort(c(contam_indices, contam_indices + 1))
asv_fasta_no_contam <- asv_fasta[- dont_want]

# haciendo una nueva tabla de conteos
asv_tab_no_contam <- asv_tab[!row.names(asv_tab) %in% contam_asvs, ]

# realizando una nueva tabla de taxonomía
asv_tax_no_contam <- asv_tax[!row.names(asv_tax) %in% contam_asvs, ]

## y ahora escribiéndolo a nuevos archivos
write(asv_fasta_no_contam, "ASVs-no-contam.fa")

write.table(asv_tab_no_contam, "ASVs_counts-no-contam.tsv", sep="\t", quote=F, col.names=NA)
write.table(asv_tax_no_contam, "ASVs_taxonomy-no-contam.tsv", sep="\t", quote=F, col.names=NA)

## 5. Calculating Diversity

#### Setup

In [ ]:
%%R

library(tidyverse)
library(phyloseq)
library(vegan)
library(DESeq2)
library(dendextend)
library(viridis)

Estas son las librerías necesarias. Ahora, vamos a alistar nuestras tablas.

In [ ]:
%%R

write(asv_fasta_no_contam, "ASVs-no-contam.fa")
write.table(asv_tab_no_contam, "ASVs_counts-no-contam.tsv", sep="\t", quote=F, col.names=NA)
write.table(asv_tax_no_contam, "ASVs_taxonomy-no-contam.tsv", sep="\t", quote=F, col.names=NA)

#ATENCION: Esto remueve todos tus archivos. Asegúrate de que tienes todo.

rm(list=ls())
count_tab <- read.table("ASVs_counts-no-contam.tsv", header=T, row.names=1, check.names=F, sep="\t")[ , -c(1:4)]

Momento ahora de realizar nuestra tabla de metadata:

In [ ]:
%%R

tax_tab <- as.matrix(read.table("ASVs_taxonomy-no-contam.tsv", header=T, row.names=1, check.names=F, sep="\t"))
sample_info_tab <- read.table("sample_info.tsv", header=T, row.names=1, check.names=F, sep="\t")

#Vamos a necesitar esta columna pronto

sample_info_tab$color <- as.character(sample_info_tab$color)
sample_info_tab # checa

Ahora, con esto nos falta normalizar nuestra data. Para esto hay varias técnicas. Nosotros nos valdremos de un paquete que puede realizar esta función para nosotros sin necesidad de recurrir a técnicas misteriosas:

In [ ]:
%%R

deseq_counts <- DESeqDataSetFromMatrix(count_tab, colData = sample_info_tab, design = ~type)
deseq_counts_vst <- varianceStabilizingTransformation(deseq_counts)

# ATENCIÓN por si figura un error:
# deseq_counts <- estimateSizeFactors(deseq_counts, type = "poscounts")
# deseq_counts_vst <- varianceStabilizingTransformation(deseq_counts)

vst_trans_count_tab <- assay(deseq_counts_vst)
euc_dist <- dist(t(vst_trans_count_tab))
#Esto permite calcular la distancia euclidiana que requerimos para nuestra primera evaluación de beta-diversidad

### Beta-diversity

Ya con las distancias hechas, estamos listos para dibujar nuestra primera gráfica, que vendría a ser la siguiente:

In [ ]:
%%R

euc_clust <- hclust(euc_dist, method="ward.D2")

#Se puede plotear de este modo
plot(euc_clust)

#Como también podemos transformarlo en un dendograma, que es más simple de editar
euc_dend <- as.dendrogram(euc_clust, hang=0.1)
dend_cols <- as.character(sample_info_tab$color[order.dendrogram(euc_dend)])
labels_colors(euc_dend) <- dend_cols

plot(euc_dend, ylab="VST Euc. dist.")

Y ahora, por qué no aplicar un PCoA. Este tipo de gráfica nos permitirá ver una posible clusterización. Esto lo podremos lograr de la siguiente forma:

In [ ]:
%%R

#Phyloseq funciona otorgándole un objeto que se forma con las tablas
#hechas previamente. Así que hay que ponerlas de la siguiente forma:
vst_count_phy <- otu_table(vst_trans_count_tab, taxa_are_rows=T)
sample_info_tab_phy <- sample_data(sample_info_tab)
vst_physeq <- phyloseq(vst_count_phy, sample_info_tab_phy)

# Generamos el PCoA de la siguiente forma.
vst_pcoa <- ordinate(vst_physeq, method="MDS", distance="euclidean")
eigen_vals <- vst_pcoa$values$Eigenvalues #escalamiento de ejes

plot_ordination(vst_physeq, vst_pcoa, color="char") 
+ geom_point(size=1) 
+ labs(col="type") 
+ geom_text(aes(label=rownames(sample_info_tab), hjust=0.3, vjust=-0.4)) 
+ coord_fixed(sqrt(eigen_vals[2]/eigen_vals[1])) 
+ ggtitle("PCoA") 
+ scale_color_manual(values=unique(sample_info_tab$color[order(sample_info_tab$char)])) 
+ theme_bw() 
+ theme(legend.position="none")

### Alfa-diversity

Y ahora, pasaremos a ver la alfa diversidad de nuestras muestras. Aquí usaremos diversidad Shannon y Chao1:

In [ ]:
%%R

#first we need to create a phyloseq object using our un-transformed count table

count_tab_phy <- otu_table(count_tab, taxa_are_rows=T)
tax_tab_phy <- tax_table(tax_tab)

ASV_physeq <- phyloseq(count_tab_phy, tax_tab_phy, sample_info_tab_phy)

#and now we can call the plot_richness() function on our phyloseq object

plot_richness(ASV_physeq, color="char", measures=c("Chao1", "Shannon")) 
+ scale_color_manual(values=unique(sample_info_tab$color[order(sample_info_tab$char)])) 
+ theme_bw() + theme(legend.title = element_blank(), axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 1))

También podemos tener otra forma de presentación para representar la riqueza.

In [ ]:
%%R

plot_richness(ASV_physeq, x="type", color="char", measures=c("Chao1", "Shannon")) 
+ scale_color_manual(values=unique(sample_info_tab$color[order(sample_info_tab$char)])) 
+ theme_bw() + theme(legend.title = element_blank(), axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 1))

## 5. Visualizing the taxonomy

Ahora, podemos visualizar los grupos taxonómicos que hemos obtenido. Para ello, habrá que alistar los grupos taxonómicos que tenemos.

Haremos una tabla que sume a los phylums:

In [ ]:
%%R

phyla_counts_tab <- otu_table(tax_glom(ASV_physeq, taxrank="phylum"))

Un vector que pueda darnos los nombres de filas:

In [ ]:
%%R

phyla_tax_vec <- as.vector(tax_table(tax_glom(ASV_physeq, taxrank="phylum"))[,"phylum"])
rownames(phyla_counts_tab) <- as.vector(phyla_tax_vec)

Es momento de ordenar y tomar en cuenta a los no-clasificados:

In [ ]:
%%R

unclassified_tax_counts <- colSums(count_tab) -
colSums(phyla_counts_tab)

Y ahora habrá que añadir esto a la tabla:

In [ ]:
%%R

phyla_and_unidentified_counts_tab <- rbind(phyla_counts_tab, "Unclassified"=unclassified_tax_counts)

Ahora nos deshacemos de *Proteobacteria*. No worries, lo agregaremos después cuando hagamos algo por clases:

In [ ]:
%%R

temp_major_taxa_counts_tab <- phyla_and_unidentified_counts_tab[!row.names(phyla_and_unidentified_counts_tab) %in% "Proteobacteria", ]

Y ahora haremos una tabla por clases:

In [ ]:
%%R

class_counts_tab <- otu_table(tax_glom(ASV_physeq, taxrank="class"))

phy_tmp_vec <- class_tax_phy_tab[,2]
class_tmp_vec <- class_tax_phy_tab[,3]
rows_tmp <- row.names(class_tax_phy_tab)
class_tax_tab <- data.frame("phylum"=phy_tmp_vec, "class"=class_tmp_vec, row.names = rows_tmp)

# Un vector que tenga todas las clases de Proteobacteria
proteo_classes_vec <- as.vector(class_tax_tab[class_tax_tab$phylum == "Proteobacteria", "class"])

# Cambiando nombres
rownames(class_counts_tab) <- as.vector(class_tax_tab$class)

# Conteo de clases de proteobacterias
proteo_class_counts_tab <- class_counts_tab[row.names(class_counts_tab) %in% proteo_classes_vec, ]

#Aquellas que no clasificaron, hay que ponerlas en algún lado
proteo_no_class_annotated_counts <- phyla_and_unidentified_counts_tab[row.names(phyla_and_unidentified_counts_tab) %in% "Proteobacteria", ] - colSums(proteo_class_counts_tab)

#Ahora los combinaremos
major_taxa_counts_tab <- rbind(temp_major_taxa_counts_tab, proteo_class_counts_tab, "Unresolved_Proteobacteria"=proteo_no_class_annotated_counts)

# Para ver que no hayamos perdido info, comparamos. Si es TRUE, bien
identical(colSums(major_taxa_counts_tab), colSums(count_tab))

# tabla de proporciones
major_taxa_proportions_tab <- apply(major_taxa_counts_tab, 2, function(x) x/sum(x)*100)

# Chequeemos la dimensión
dim(major_taxa_proportions_tab)

# Filtremos aquellos grupos con menos del 5%
temp_filt_major_taxa_proportions_tab <-
data.frame(major_taxa_proportions_tab[apply(major_taxa_proportions_tab, 1, max) > 5, ])

# Después del filtrado
dim(temp_filt_major_taxa_proportions_tab)

#Creamos el grupo “other” para no perder la cabeza
filtered_proportions <- colSums(major_taxa_proportions_tab) -
colSums(temp_filt_major_taxa_proportions_tab)
filt_major_taxa_proportions_tab <- rbind(temp_filt_major_taxa_proportions_tab, "Other"=filtered_proportions)

# copia para editar sin remordimiento
filt_major_taxa_proportions_tab_for_plot <- filt_major_taxa_proportions_tab

#columna de nombres de taxa
filt_major_taxa_proportions_tab_for_plot$Major_Taxa <- row.names(filt_major_taxa_proportions_tab_for_plot)

# transformamos la tabla para que sea más fácil de graficar
filt_major_taxa_proportions_tab_for_plot.g <- pivot_longer(filt_major_taxa_proportions_tab_for_plot, !Major_Taxa, names_to = "Sample", values_to = "Proportion") %>% data.frame()

# Añadimos columnas de color y características
sample_info_for_merge<-data.frame("Sample"=row.names(sample_info_tab), "char"=sample_info_tab$char, "color"=sample_info_tab$color, stringsAsFactors=F)

#Añadimos esto a nuestra tabla hecha para graficar
filt_major_taxa_proportions_tab_for_plot.g2 <- merge(filt_major_taxa_proportions_tab_for_plot.g, sample_info_for_merge)

# Y ahora ggplot
ggplot(filt_major_taxa_proportions_tab_for_plot.g2, aes(x=Sample, y=Proportion, fill=Major_Taxa)) +
geom_bar(width=0.6, stat="identity") +
theme_bw() +
theme(axis.text.x=element_text(angle=90, vjust=0.4, hjust=1), legend.title=element_blank()) +
labs(x="Sample", y="% of 16S rRNA gene copies recovered", title="All samples")